In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("인천시_전기차_충전소_입지분석_250M_격자별_전처리_완료.csv", encoding = "CP949")
df.head(1)

,gid,경제인구,건물2,건물3,변압기,전기차,서비스기간,서비스빈도,서비스일수,총충전량,충전소,건물1,민원행정기관,주차장,도서관
0,가사46aa99bb,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0


# 조건 : 충전소가 있는 격자들만 추출!  
## (과정 : 충전소가 있는 격자들만 추출하여 선형회귀식을 만든 후
##             충전소가 없는 격자에 대입하여 충전소가 있을 경우 평균충전량에 
##             대한 추정치를 구한 후 순위를 매겨 상위권에 있는 격자에 입지선정 실시.)


# 종속변수 : 총충전량 / 일수  or 빈도 or 기간 

# 독립변수 : 전기차, 변압기, 주차장, 건물1~3, 행정기관, 도서관, 경제인구

###  전처리

In [3]:
# 복사
ch_tot_info_temp = df.copy()
ch_tot_info_temp.head(30)

,gid,경제인구,건물2,건물3,변압기,전기차,서비스기간,서비스빈도,서비스일수,총충전량,충전소,건물1,민원행정기관,주차장,도서관
0,가사46aa99bb,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,가사46ab99ba,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,가사46ab99bb,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,가사46ba99ab,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,가사46ba99ba,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
5,가사46ba99bb,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
6,가사46bb99ab,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
7,가사46bb99ba,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
8,가사46bb99bb,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2,0,0,0
9,가사47aa99aa,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [4]:
ch_tot_info_temp.isnull().sum()

gid           0
경제인구      14949
건물2           0
건물3           0
변압기           0
전기차           0
서비스기간     19664
서비스빈도     19664
서비스일수     19664
총충전량      19664
충전소       19664
건물1           0
민원행정기관        0
주차장           0
도서관           0
dtype: int64

In [5]:
# 데이터 보고 columns 순서를 재설정해야함.
#ch_tot_info_temp.loc[:,['서비스기간', '서비스빈도', '서비스일수', '총충전량']] = ch_tot_info_temp.loc[:,['서비스기간', '서비스빈도', '서비스일수', '총충전량']].fillna(0)
ch_tot_info_temp = ch_tot_info_temp.fillna(0)
ch_tot_info_temp

,gid,경제인구,건물2,건물3,변압기,전기차,서비스기간,서비스빈도,서비스일수,총충전량,충전소,건물1,민원행정기관,주차장,도서관
0,가사46aa99bb,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1,가사46ab99ba,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2,가사46ab99bb,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
3,가사46ba99ab,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
4,가사46ba99ba,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
5,가사46ba99bb,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
6,가사46bb99ab,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
7,가사46bb99ba,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
8,가사46bb99bb,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2,0,0,0
9,가사47aa99aa,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [6]:
ch_tot_info_temp.iloc[:,2].unique()

array([ 0,  2,  1,  3,  5,  9,  4, 22, 13,  6,  7, 24, 12, 14, 16, 23,  8,
       10, 11, 20, 18, 15, 17, 19, 21, 25, 29, 33, 36, 35, 30, 26, 51, 27,
       31], dtype=int64)

In [7]:
list(ch_tot_info_temp.columns)[5]

'전기차'

In [8]:
ch_tot_info_temp['충전소'].unique()

array([0., 1., 2.])

In [9]:
ch_tot_info_temp.sort_values(['충전소'], ascending = False).head(15)

,gid,경제인구,건물2,건물3,변압기,전기차,서비스기간,서비스빈도,서비스일수,총충전량,충전소,건물1,민원행정기관,주차장,도서관
15106,다사23bb48bb,1620.0,5,0,4,5,992.0,1197.0,619.0,21410.66,2.0,0,1,0,1
15045,다사23bb33ba,1689.0,8,0,2,4,889.0,813.0,524.0,12122.82,2.0,2,0,0,0
14673,다사22bb48aa,775.0,8,0,2,0,446.0,76.0,65.0,865.20,2.0,1,1,0,0
15303,다사24ab39ba,988.0,6,0,2,0,458.0,851.0,375.0,16264.42,1.0,5,0,0,0
18645,다사31bb33ba,1873.0,5,0,1,1,493.0,288.0,221.0,8934.92,1.0,0,0,0,0
18646,다사31bb33bb,1161.0,4,0,1,2,482.0,546.0,354.0,6822.66,1.0,1,0,0,0
14962,다사23ba40ab,472.0,3,0,3,0,346.0,68.0,56.0,885.55,1.0,16,0,0,0
16773,다사27ab34ab,2187.0,10,0,2,1,528.0,436.0,290.0,10390.10,1.0,3,0,0,0
18973,다사32ba49aa,688.0,0,1,23,1,356.0,397.0,226.0,7999.91,1.0,26,0,0,0
13399,다사16ab44aa,1111.0,7,0,1,3,525.0,736.0,366.0,10157.98,1.0,0,0,0,0


In [10]:
len(ch_tot_info_temp.columns)

15

In [11]:
# 'gid', '건물1', '건물2', '건물3', '민원행정기관', '도서관','경제인구', 
# '주차장','전기차', '변압기', '충전소','서비스기간','서비스빈도','서비스일수','총충전량'

ctit = ch_tot_info_temp.loc[:, ['gid', '건물1', '건물2', '건물3', '민원행정기관', '도서관'
                                ,'경제인구', '주차장','전기차', '변압기', '충전소',
                                '서비스기간','서비스빈도','서비스일수','총충전량']]
ctit.head(1)

,gid,건물1,건물2,건물3,민원행정기관,도서관,경제인구,주차장,전기차,변압기,충전소,서비스기간,서비스빈도,서비스일수,총충전량
0,가사46aa99bb,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [12]:
len(ctit.columns)

15

In [13]:
for i in list(ctit.columns) :
    print(ctit[i].unique() ,"  ")

['가사46aa99bb' '가사46ab99ba' '가사46ab99bb' ... '다사37ba53ab' '다사37ba53ba'
 '다사37ba53bb']   
[  0   2   1   7   6  14   3  41   4  29  12   9  48  11  15   5  37  20
  61  16  38  76  17  25  13  19  44  40  18  21  47  24  35  68   8  10
  30  33  22  81  67  99  78  36 112  80  72  23  28  26  79  51  69  27
  32  89  42 103  60  43  52  55  58  84  31  46  45  39  64  34  49 151
 156  83  54  85  65 111  71  90  53 122  75 106 238 139  57  56  62 165
  59 101  97 113 130 199  95  98 117 126  63  50 104  66  74 109 102  77
  88 100  91 110 114  73  70  94  82  86 105 143 118  93 124 421 419 115
 127 141  92]   
[ 0  2  1  3  5  9  4 22 13  6  7 24 12 14 16 23  8 10 11 20 18 15 17 19
 21 25 29 33 36 35 30 26 51 27 31]   
[ 0  1  2  3  4  5 11  9]   
[0 1 2 3 4]   
[0 1 2 4]   
[   0.   22.   86. ... 1300.  248.  122.]   
[ 0  1  2  3  5  4  8  6  7  9 10]   
[ 0  2  1  5 10  3  4  7  9 23 14 11  6  8 22 26 24 16 30 15 32 29 12 31]   
[  0   1   3   2   4   5   6  11  15  10  19  12   8  18

 ## 평균 추가

In [14]:
ctit['총/기간'] = [0]*len(ctit)
ctit['총/빈도'] = [0]*len(ctit)
ctit['총/일수'] = [0]*len(ctit)

In [15]:
# 서비스 기간이 값이 0인 행의 index
a = set(ctit[ ctit['서비스기간'] == 0 ].index)

# 서비스 빈도가 값이 0인 행의 index
b = set(ctit[ ctit['서비스기간'] == 0 ].index)

# 서비스 일 수가 값이 0인 행의 index
c = set(ctit[ ctit['서비스기간'] == 0 ].index) 

# 위에서 구한 0인 값들 제외하고 기준에 따른 값들 구하기

ctit.loc[ list(set(ctit['서비스기간'].index) - a), '총/기간'] = (
    ctit.loc[ list(set(ctit['서비스기간'].index) - a), '총충전량'] / ctit.loc[ list(set(ctit['서비스기간'].index) - a), '서비스기간']
)
    
ctit.loc[ list(set(ctit['서비스빈도'].index) - b), '총/빈도'] = (
    ctit.loc[ list(set(ctit['서비스빈도'].index) - b), '총충전량'] / ctit.loc[ list(set(ctit['서비스빈도'].index) - b), '서비스빈도'] 
)
    
ctit.loc[ list(set(ctit['서비스일수'].index) - c), '총/일수'] = (
    ctit.loc[ list(set(ctit['서비스일수'].index) - c), '총충전량'] / ctit.loc[ list(set(ctit['서비스일수'].index) - c), '서비스일수'] 
)

ctit

,gid,건물1,건물2,건물3,민원행정기관,도서관,경제인구,주차장,전기차,변압기,충전소,서비스기간,서비스빈도,서비스일수,총충전량,총/기간,총/빈도,총/일수
0,가사46aa99bb,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,가사46ab99ba,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,가사46ab99bb,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,가사46ba99ab,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,가사46ba99ba,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,가사46ba99bb,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,가사46bb99ab,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,가사46bb99ba,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,가사46bb99bb,2,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,가사47aa99aa,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
ctit.sort_values(['충전소'], ascending = False)

,gid,건물1,건물2,건물3,민원행정기관,도서관,경제인구,주차장,전기차,변압기,충전소,서비스기간,서비스빈도,서비스일수,총충전량,총/기간,총/빈도,총/일수
15106,다사23bb48bb,0,5,0,1,1,1620.0,0,5,4,2.0,992.0,1197.0,619.0,21410.66,21.583327,17.886934,34.589111
15045,다사23bb33ba,2,8,0,0,0,1689.0,0,4,2,2.0,889.0,813.0,524.0,12122.82,13.636468,14.911218,23.135153
14673,다사22bb48aa,1,8,0,1,0,775.0,0,0,2,2.0,446.0,76.0,65.0,865.20,1.939910,11.384211,13.310769
15303,다사24ab39ba,5,6,0,0,0,988.0,0,0,2,1.0,458.0,851.0,375.0,16264.42,35.511834,19.112127,43.371787
18645,다사31bb33ba,0,5,0,0,0,1873.0,0,1,1,1.0,493.0,288.0,221.0,8934.92,18.123570,31.024028,40.429502
18646,다사31bb33bb,1,4,0,0,0,1161.0,0,2,1,1.0,482.0,546.0,354.0,6822.66,14.154896,12.495714,19.273051
14962,다사23ba40ab,16,3,0,0,0,472.0,0,0,3,1.0,346.0,68.0,56.0,885.55,2.559393,13.022794,15.813393
16773,다사27ab34ab,3,10,0,0,0,2187.0,0,1,2,1.0,528.0,436.0,290.0,10390.10,19.678220,23.830505,35.827931
18973,다사32ba49aa,26,0,1,0,0,688.0,0,1,23,1.0,356.0,397.0,226.0,7999.91,22.471657,20.150907,35.397832
13399,다사16ab44aa,0,7,0,0,0,1111.0,0,3,1,1.0,525.0,736.0,366.0,10157.98,19.348533,13.801603,27.754044


In [17]:
len(ctit[ctit['충전소'] >= 1])

125

In [18]:
len(ctit[ctit['충전소'] == 0])

19664

### 민원행정기관과 도서관을 관공서로 결합

In [20]:
df_pre = ctit.copy()

temp = df_pre['민원행정기관'] + df_pre['도서관']
df_pre['관공서'] = list(temp)
df_pre.sort_values(['충전소'], ascending = False).head(1)

,gid,건물1,건물2,건물3,민원행정기관,도서관,경제인구,주차장,전기차,변압기,충전소,서비스기간,서비스빈도,서비스일수,총충전량,총/기간,총/빈도,총/일수,관공서
15106,다사23bb48bb,0,5,0,1,1,1620.0,0,5,4,2.0,992.0,1197.0,619.0,21410.66,21.583327,17.886934,34.589111,2


## 순서 재배치 

In [21]:
df_pre2 = df_pre.iloc[:, [0,1,2,3,-1, 6,7,8,9,10,14 ,11,12,13,15,16,17]]
df_pre2.sort_values(['충전소'], ascending = False).head(10)

,gid,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량,서비스기간,서비스빈도,서비스일수,총/기간,총/빈도,총/일수
15106,다사23bb48bb,0,5,0,2,1620.0,0,5,4,2.0,21410.66,992.0,1197.0,619.0,21.583327,17.886934,34.589111
15045,다사23bb33ba,2,8,0,0,1689.0,0,4,2,2.0,12122.82,889.0,813.0,524.0,13.636468,14.911218,23.135153
14673,다사22bb48aa,1,8,0,1,775.0,0,0,2,2.0,865.20,446.0,76.0,65.0,1.939910,11.384211,13.310769
15303,다사24ab39ba,5,6,0,0,988.0,0,0,2,1.0,16264.42,458.0,851.0,375.0,35.511834,19.112127,43.371787
18645,다사31bb33ba,0,5,0,0,1873.0,0,1,1,1.0,8934.92,493.0,288.0,221.0,18.123570,31.024028,40.429502
18646,다사31bb33bb,1,4,0,0,1161.0,0,2,1,1.0,6822.66,482.0,546.0,354.0,14.154896,12.495714,19.273051
14962,다사23ba40ab,16,3,0,0,472.0,0,0,3,1.0,885.55,346.0,68.0,56.0,2.559393,13.022794,15.813393
16773,다사27ab34ab,3,10,0,0,2187.0,0,1,2,1.0,10390.10,528.0,436.0,290.0,19.678220,23.830505,35.827931
18973,다사32ba49aa,26,0,1,0,688.0,0,1,23,1.0,7999.91,356.0,397.0,226.0,22.471657,20.150907,35.397832
13399,다사16ab44aa,0,7,0,0,1111.0,0,3,1,1.0,10157.98,525.0,736.0,366.0,19.348533,13.801603,27.754044


In [22]:
charge_over_1 = df_pre2[ df_pre2['충전소'] >=1 ]
charge_over_1.head(1)

,gid,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량,서비스기간,서비스빈도,서비스일수,총/기간,총/빈도,총/일수
6525,다사02bb38bb,12,0,0,3,49.0,0,0,0,1.0,15565.04,520.0,881.0,407.0,29.932769,17.667469,38.243342


 # 4가지 종속변수와 독립변수들 간의 상관관계

### 충전소가 존재하는 격자들의 총충전량과 독립변수들 간의 상관관계

In [39]:
charge_df = charge_over_1.iloc[:, [10,1,2,3,4,5,6,7,8,9]]
charge_df.corr()

,총충전량,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
총충전량,1.000000,-0.148862,-0.210180,0.021782,-0.019836,-0.166938,-0.037266,0.442906,-0.114887,0.058776
건물1,-0.148862,1.000000,-0.187666,0.227609,0.106516,-0.124472,0.279138,-0.161426,0.567417,-0.096879
건물2,-0.210180,-0.187666,1.000000,-0.183614,-0.191521,0.642280,-0.085035,0.046994,-0.134540,0.041645
건물3,0.021782,0.227609,-0.183614,1.000000,0.072079,-0.200555,0.060922,-0.083419,0.346254,-0.043042
관공서,-0.019836,0.106516,-0.191521,0.072079,1.000000,-0.117863,0.129342,-0.062467,0.094469,0.092965
경제인구,-0.166938,-0.124472,0.642280,-0.200555,-0.117863,1.000000,-0.131837,0.226875,-0.049778,0.040548
주차장,-0.037266,0.279138,-0.085035,0.060922,0.129342,-0.131837,1.000000,-0.038783,0.202240,-0.058157
전기차,0.442906,-0.161426,0.046994,-0.083419,-0.062467,0.226875,-0.038783,1.000000,-0.108407,0.062137
변압기,-0.114887,0.567417,-0.134540,0.346254,0.094469,-0.049778,0.202240,-0.108407,1.000000,-0.069496
충전소,0.058776,-0.096879,0.041645,-0.043042,0.092965,0.040548,-0.058157,0.062137,-0.069496,1.000000


### 충전소가 존재하는 격자들의 총/기간과 독립변수들 간의 상관관계

In [46]:
charge_df2 = charge_over_1.iloc[:, [14,1,2,3,4,5,6,7,8,9]]
charge_df2.corr()

,총/기간,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
총/기간,1.000000,-0.122374,-0.245028,0.065862,-0.027401,-0.226827,-0.035325,0.389441,-0.103701,-0.045316
건물1,-0.122374,1.000000,-0.187666,0.227609,0.106516,-0.124472,0.279138,-0.161426,0.567417,-0.096879
건물2,-0.245028,-0.187666,1.000000,-0.183614,-0.191521,0.642280,-0.085035,0.046994,-0.134540,0.041645
건물3,0.065862,0.227609,-0.183614,1.000000,0.072079,-0.200555,0.060922,-0.083419,0.346254,-0.043042
관공서,-0.027401,0.106516,-0.191521,0.072079,1.000000,-0.117863,0.129342,-0.062467,0.094469,0.092965
경제인구,-0.226827,-0.124472,0.642280,-0.200555,-0.117863,1.000000,-0.131837,0.226875,-0.049778,0.040548
주차장,-0.035325,0.279138,-0.085035,0.060922,0.129342,-0.131837,1.000000,-0.038783,0.202240,-0.058157
전기차,0.389441,-0.161426,0.046994,-0.083419,-0.062467,0.226875,-0.038783,1.000000,-0.108407,0.062137
변압기,-0.103701,0.567417,-0.134540,0.346254,0.094469,-0.049778,0.202240,-0.108407,1.000000,-0.069496
충전소,-0.045316,-0.096879,0.041645,-0.043042,0.092965,0.040548,-0.058157,0.062137,-0.069496,1.000000


### 충전소가 존재하는 격자들의 총/빈도와 독립변수들 간의 상관관계

In [47]:
charge_df3 = charge_over_1.iloc[:, [15,1,2,3,4,5,6,7,8,9]]
charge_df3.corr()

,총/빈도,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
총/빈도,1.000000,-0.188963,-0.191937,-0.003499,0.022491,-0.163554,-0.050566,0.109706,-0.212050,-0.054396
건물1,-0.188963,1.000000,-0.187666,0.227609,0.106516,-0.124472,0.279138,-0.161426,0.567417,-0.096879
건물2,-0.191937,-0.187666,1.000000,-0.183614,-0.191521,0.642280,-0.085035,0.046994,-0.134540,0.041645
건물3,-0.003499,0.227609,-0.183614,1.000000,0.072079,-0.200555,0.060922,-0.083419,0.346254,-0.043042
관공서,0.022491,0.106516,-0.191521,0.072079,1.000000,-0.117863,0.129342,-0.062467,0.094469,0.092965
경제인구,-0.163554,-0.124472,0.642280,-0.200555,-0.117863,1.000000,-0.131837,0.226875,-0.049778,0.040548
주차장,-0.050566,0.279138,-0.085035,0.060922,0.129342,-0.131837,1.000000,-0.038783,0.202240,-0.058157
전기차,0.109706,-0.161426,0.046994,-0.083419,-0.062467,0.226875,-0.038783,1.000000,-0.108407,0.062137
변압기,-0.212050,0.567417,-0.134540,0.346254,0.094469,-0.049778,0.202240,-0.108407,1.000000,-0.069496
충전소,-0.054396,-0.096879,0.041645,-0.043042,0.092965,0.040548,-0.058157,0.062137,-0.069496,1.000000


### 충전소가 존재하는 격자들의 총/일수와 독립변수들 간의 상관관계

In [48]:
charge_df4 = charge_over_1.iloc[:, [16,1,2,3,4,5,6,7,8,9]]
charge_df4.corr()

,총/일수,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
총/일수,1.000000,-0.167308,-0.258861,0.062593,-0.003787,-0.227429,-0.064333,0.337883,-0.155849,-0.054050
건물1,-0.167308,1.000000,-0.187666,0.227609,0.106516,-0.124472,0.279138,-0.161426,0.567417,-0.096879
건물2,-0.258861,-0.187666,1.000000,-0.183614,-0.191521,0.642280,-0.085035,0.046994,-0.134540,0.041645
건물3,0.062593,0.227609,-0.183614,1.000000,0.072079,-0.200555,0.060922,-0.083419,0.346254,-0.043042
관공서,-0.003787,0.106516,-0.191521,0.072079,1.000000,-0.117863,0.129342,-0.062467,0.094469,0.092965
경제인구,-0.227429,-0.124472,0.642280,-0.200555,-0.117863,1.000000,-0.131837,0.226875,-0.049778,0.040548
주차장,-0.064333,0.279138,-0.085035,0.060922,0.129342,-0.131837,1.000000,-0.038783,0.202240,-0.058157
전기차,0.337883,-0.161426,0.046994,-0.083419,-0.062467,0.226875,-0.038783,1.000000,-0.108407,0.062137
변압기,-0.155849,0.567417,-0.134540,0.346254,0.094469,-0.049778,0.202240,-0.108407,1.000000,-0.069496
충전소,-0.054050,-0.096879,0.041645,-0.043042,0.092965,0.040548,-0.058157,0.062137,-0.069496,1.000000


 # 4가지 종속변수로 회귀분석 실시

### 총충전량 ~ 모든변수 회귀분석

In [31]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.ols( formula = "총충전량 ~ " + ("+".join(list(charge_df.columns)[1:])), data = charge_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   총충전량   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     5.321
Date:                Thu, 07 Nov 2019   Prob (F-statistic):           4.58e-06
Time:                        09:30:53   Log-Likelihood:                -1289.1
No. Observations:                 125   AIC:                             2598.
Df Residuals:                     115   BIC:                             2627.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8627.9963   4882.011      1.767      0.080   -1042.327    1.83e+04
건물1          -56.7041     58.763     -0.965      0.337    -173.101      59.693
건물2         -252.7705    218.383     -1.157      0.249    -685.344     179.803
건물3          649.9437   1493.183      0.435      0.664   -2307.764    3607.651
관공서         -270.7914    790.483     -0.343      0.733   -1836.586    1295.003
경제인구          -2.5198      1.297     -1.943      0.054      -5.089       0.049
주차장         -328.7707   1448.662     -0.227      0.821   -3198.292    2540.750
전기차         1581.6202    274.889      5.754      0.000    1037.119    2126.122
변압기          -83.9216    207.543     -0.404      0.687    -495.024     327.181
충전소         1922.1377   4501.641      0.427      0.670   -6994.747    1.08e+04
==============================================================================
Omnibus:                       58.399   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              206.676
Skew:                           1.696   Prob(JB):                     1.32e-45
Kurtosis:                       8.308   Cond. No.                     1.32e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [49]:
charge_df2.rename( columns = { 
                        '총/기간' : '평균_기간'}, inplace = True)
charge_df3.rename( columns = { 
                        '총/빈도' : '평균_빈도'}, inplace = True)
charge_df4.rename( columns = { 
                        '총/일수' : '평균_일수'}, inplace = True)

### 총/기간 ~ 모든변수 회귀분석

In [51]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


model2 = smf.ols( formula = "평균_기간 ~ " + ("+".join(list(charge_df2.columns)[1:])), data = charge_df2)
result2 = model2.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  평균_기간   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     5.064
Date:                Thu, 07 Nov 2019   Prob (F-statistic):           9.29e-06
Time:                        09:40:31   Log-Likelihood:                -519.79
No. Observations:                 125   AIC:                             1060.
Df Residuals:                     115   BIC:                             1088.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     32.5380     10.368      3.138      0.002      12.000      53.076
건물1           -0.1042      0.125     -0.835      0.405      -0.351       0.143
건물2           -0.5148      0.464     -1.110      0.269      -1.434       0.404
건물3            2.5723      3.171      0.811      0.419      -3.709       8.854
관공서           -0.6872      1.679     -0.409      0.683      -4.013       2.638
경제인구          -0.0065      0.003     -2.349      0.021      -0.012      -0.001
주차장           -1.1425      3.077     -0.371      0.711      -7.237       4.952
전기차            3.0888      0.584      5.291      0.000       1.932       4.245
변압기           -0.2492      0.441     -0.565      0.573      -1.122       0.624
충전소           -7.7739      9.561     -0.813      0.418     -26.712      11.164
==============================================================================
Omnibus:                       57.666   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              181.876
Skew:                           1.734   Prob(JB):                     3.21e-40
Kurtosis:                       7.785   Cond. No.                     1.32e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 총/빈도 ~ 모든변수 회귀분석

In [52]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


model3 = smf.ols( formula = "평균_빈도 ~ " + ("+".join(list(charge_df3.columns)[1:])), data = charge_df3)
result3 = model3.fit()
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  평균_빈도   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     1.849
Date:                Thu, 07 Nov 2019   Prob (F-statistic):             0.0667
Time:                        09:41:50   Log-Likelihood:                -425.97
No. Observations:                 125   AIC:                             871.9
Df Residuals:                     115   BIC:                             900.2
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     25.8363      4.895      5.278      0.000      16.141      35.532
건물1           -0.0719      0.059     -1.220      0.225      -0.189       0.045
건물2           -0.3234      0.219     -1.477      0.142      -0.757       0.110
건물3            0.6545      1.497      0.437      0.663      -2.311       3.620
관공서            0.1866      0.793      0.235      0.814      -1.383       1.756
경제인구          -0.0010      0.001     -0.738      0.462      -0.004       0.002
주차장           -0.1642      1.452     -0.113      0.910      -3.041       2.713
전기차            0.3206      0.276      1.163      0.247      -0.225       0.866
변압기           -0.3225      0.208     -1.550      0.124      -0.735       0.090
충전소           -3.8775      4.513     -0.859      0.392     -12.818       5.063
==============================================================================
Omnibus:                        0.227   Durbin-Watson:                   1.626
Prob(Omnibus):                  0.893   Jarque-Bera (JB):                0.384
Skew:                          -0.071   Prob(JB):                        0.825
Kurtosis:                       2.769   Cond. No.                     1.32e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 총/일수 ~ 모든변수 회귀분석

In [53]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


model4 = smf.ols( formula = "평균_일수 ~ " + ("+".join(list(charge_df4.columns)[1:])), data = charge_df4)
result4 = model4.fit()
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  평균_일수   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     4.620
Date:                Thu, 07 Nov 2019   Prob (F-statistic):           3.18e-05
Time:                        09:41:50   Log-Likelihood:                -523.90
No. Observations:                 125   AIC:                             1068.
Df Residuals:                     115   BIC:                             1096.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     49.0478     10.715      4.578      0.000      27.824      70.272
건물1           -0.1496      0.129     -1.160      0.249      -0.405       0.106
건물2           -0.7104      0.479     -1.482      0.141      -1.660       0.239
건물3            3.1727      3.277      0.968      0.335      -3.319       9.664
관공서           -0.1436      1.735     -0.083      0.934      -3.580       3.293
경제인구          -0.0056      0.003     -1.964      0.052      -0.011    4.84e-05
주차장           -1.7577      3.179     -0.553      0.581      -8.056       4.540
전기차            2.6782      0.603      4.439      0.000       1.483       3.873
변압기           -0.4746      0.456     -1.042      0.300      -1.377       0.428
충전소           -9.6310      9.880     -0.975      0.332     -29.201       9.939
==============================================================================
Omnibus:                       24.202   Durbin-Watson:                   1.606
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.788
Skew:                           0.962   Prob(JB):                     1.69e-08
Kurtosis:                       4.781   Cond. No.                     1.32e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
### 총/기간 ~ 모든변수 회귀분석

import statsmodels.api as sm
import statsmodels.formula.api as smf


model2 = smf.ols( formula = "평균_기간 ~ " + ("+".join(list(charge_df2.columns)[1:])), data = charge_df2)
result2 = model2.fit()
result2.summary()

# 상관계수

# 1. 가점 고려 안함 

In [19]:
df_pre_cor = df_pre2.iloc[:, 1:11 ]
df_pre_cor.sort_values(['충전소'], ascending = False).head(15)

,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량
15106,0,5,0,2,1620.0,0,5,4,2.0,21410.66
15045,2,8,0,0,1689.0,0,4,2,2.0,12122.82
14673,1,8,0,1,775.0,0,0,2,2.0,865.20
15303,5,6,0,0,988.0,0,0,2,1.0,16264.42
18645,0,5,0,0,1873.0,0,1,1,1.0,8934.92
18646,1,4,0,0,1161.0,0,2,1,1.0,6822.66
14962,16,3,0,0,472.0,0,0,3,1.0,885.55
16773,3,10,0,0,2187.0,0,1,2,1.0,10390.10
18973,26,0,1,0,688.0,0,1,23,1.0,7999.91
13399,0,7,0,0,1111.0,0,3,1,1.0,10157.98


In [20]:
df_pre_cor.corr()

,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량
건물1,1.000000,0.227850,0.093520,0.219049,0.345968,0.276262,0.093980,0.620254,0.034382,0.015005
건물2,0.227850,1.000000,0.063885,0.272091,0.813283,0.141498,0.219680,0.282194,0.189866,0.108173
건물3,0.093520,0.063885,1.000000,0.144755,0.080992,0.022758,0.128399,0.118103,0.054780,0.042572
관공서,0.219049,0.272091,0.144755,1.000000,0.339491,0.138624,0.163419,0.210363,0.113335,0.072988
경제인구,0.345968,0.813283,0.080992,0.339491,1.000000,0.244949,0.269278,0.362586,0.216660,0.131937
주차장,0.276262,0.141498,0.022758,0.138624,0.244949,1.000000,0.069037,0.193142,0.032903,0.020357
전기차,0.093980,0.219680,0.128399,0.163419,0.269278,0.069037,1.000000,0.170574,0.194253,0.219734
변압기,0.620254,0.282194,0.118103,0.210363,0.362586,0.193142,0.170574,1.000000,0.077291,0.046553
충전소,0.034382,0.189866,0.054780,0.113335,0.216660,0.032903,0.194253,0.077291,1.000000,0.685273
총충전량,0.015005,0.108173,0.042572,0.072988,0.131937,0.020357,0.219734,0.046553,0.685273,1.000000


# 2. 가점 고려 함 

In [21]:
df_pre3 = df_pre2.copy()
df_pre3['건물2'] = df_pre3['건물2'] * 2
df_pre3['건물3'] = df_pre3['건물3'] * 3
df_pre3.sort_values(['충전소'], ascending = False).head(1)

,gid,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량,서비스기간,서비스빈도,서비스일수,총/기간,총/빈도,총/일수
15106,다사23bb48bb,0,10,0,2,1620.0,0,5,4,2.0,21410.66,992.0,1197.0,619.0,21.583327,17.886934,34.589111


In [22]:
df_pre_cor2 = df_pre3.iloc[:, 1:11 ]
df_pre_cor2.sort_values(['충전소'], ascending = False).head(15)

,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량
15106,0,10,0,2,1620.0,0,5,4,2.0,21410.66
15045,2,16,0,0,1689.0,0,4,2,2.0,12122.82
14673,1,16,0,1,775.0,0,0,2,2.0,865.20
15303,5,12,0,0,988.0,0,0,2,1.0,16264.42
18645,0,10,0,0,1873.0,0,1,1,1.0,8934.92
18646,1,8,0,0,1161.0,0,2,1,1.0,6822.66
14962,16,6,0,0,472.0,0,0,3,1.0,885.55
16773,3,20,0,0,2187.0,0,1,2,1.0,10390.10
18973,26,0,3,0,688.0,0,1,23,1.0,7999.91
13399,0,14,0,0,1111.0,0,3,1,1.0,10157.98


In [23]:
df_pre_cor2.corr()

,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량
건물1,1.000000,0.227850,0.093520,0.219049,0.345968,0.276262,0.093980,0.620254,0.034382,0.015005
건물2,0.227850,1.000000,0.063885,0.272091,0.813283,0.141498,0.219680,0.282194,0.189866,0.108173
건물3,0.093520,0.063885,1.000000,0.144755,0.080992,0.022758,0.128399,0.118103,0.054780,0.042572
관공서,0.219049,0.272091,0.144755,1.000000,0.339491,0.138624,0.163419,0.210363,0.113335,0.072988
경제인구,0.345968,0.813283,0.080992,0.339491,1.000000,0.244949,0.269278,0.362586,0.216660,0.131937
주차장,0.276262,0.141498,0.022758,0.138624,0.244949,1.000000,0.069037,0.193142,0.032903,0.020357
전기차,0.093980,0.219680,0.128399,0.163419,0.269278,0.069037,1.000000,0.170574,0.194253,0.219734
변압기,0.620254,0.282194,0.118103,0.210363,0.362586,0.193142,0.170574,1.000000,0.077291,0.046553
충전소,0.034382,0.189866,0.054780,0.113335,0.216660,0.032903,0.194253,0.077291,1.000000,0.685273
총충전량,0.015005,0.108173,0.042572,0.072988,0.131937,0.020357,0.219734,0.046553,0.685273,1.000000


# 건물 가점을 곱한 다음 상관계수를 할 것인지 

# 가점을 고려한 값 아닌 그냥 카운트 수로 할 것인지 생각 ( 메뉴얼 정독!)

 # 데이터 사용하여 회귀분석 summary 및 vif 등 구해서 통계적 유의미 확인하기

In [25]:
df_reg = df_pre_cor2.iloc[:, [-1, 0,1,2,3,4,5,6,7,8]]
df_reg.head(1)

,총충전량,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
0,0.0,0,0,0,0,0.0,0,0,0,0.0


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
#from statsmodels.sandbox.regression.predstd import wls\_prediction\_std

## Y : 총 충전량

In [30]:
model = smf.ols(formula = '총충전량 ~ 건물1 + 건물2 + 건물3 + 관공서 + 경제인구 + 주차장 + 전기차 \
                  + 변압기 + 충전소', data = df_reg)

result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   총충전량   R-squared:                       0.479
Model:                            OLS   Adj. R-squared:                  0.479
Method:                 Least Squares   F-statistic:                     2022.
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        09:08:00   Log-Likelihood:            -1.5727e+05
No. Observations:               19789   AIC:                         3.146e+05
Df Residuals:                   19779   BIC:                         3.146e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.8140      5.218      0.539      0.590      -7.413      13.041
건물1            0.0182      0.501      0.036      0.971      -0.964       1.001
건물2           -5.3298      1.893     -2.815      0.005      -9.041      -1.618
건물3           -4.8065      9.416     -0.510      0.610     -23.263      13.650
관공서          -18.1582     17.636     -1.030      0.303     -52.726      16.409
경제인구          -0.0396      0.023     -1.693      0.090      -0.085       0.006
주차장            3.2996     15.387      0.214      0.830     -26.861      33.460
전기차          124.5276      6.708     18.563      0.000     111.378     137.677
변압기           -2.3398      1.793     -1.305      0.192      -5.853       1.174
충전소         7804.7929     61.529    126.848      0.000    7684.191    7925.395
==============================================================================
Omnibus:                    45601.136   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        984695453.040
Skew:                          22.027   Prob(JB):                         0.00
Kurtosis:                    1094.922   Cond. No.                     5.12e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
df_pre2.head(1)

,gid,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량,서비스기간,서비스빈도,서비스일수,총/기간,총/빈도,총/일수
0,가사46aa99bb,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Y : 평균충전량(일수 기준)

In [32]:
df_reg2 = df_pre2.iloc[:, [16,1,2,3,4,5,6,7,8,9]]
df_reg2.head(1)

,총/일수,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
0,0.0,0,0,0,0,0.0,0,0,0,0.0


In [46]:
df_reg2.rename( columns = {
    '총/일수' : '평균'
}, inplace = True)
df_reg2.head(1)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [48]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model2 = smf.ols(formula = '평균 ~ 건물1 + 건물2 + 건물3 + 관공서 + 경제인구 + 주차장 + 전기차 \
                  + 변압기 + 충전소', data = df_reg2)

result2 = model2.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     평균   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     5171.
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        09:20:02   Log-Likelihood:                -36549.
No. Observations:               19789   AIC:                         7.312e+04
Df Residuals:                   19779   BIC:                         7.320e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0110      0.012      0.943      0.346      -0.012       0.034
건물1        -2.166e-05      0.001     -0.019      0.985      -0.002       0.002
건물2           -0.0226      0.008     -2.667      0.008      -0.039      -0.006
건물3            0.0878      0.063      1.386      0.166      -0.036       0.212
관공서           -0.0036      0.040     -0.090      0.928      -0.081       0.074
경제인구       -8.442e-05   5.24e-05     -1.610      0.107      -0.000    1.84e-05
주차장            0.0021      0.035      0.061      0.951      -0.066       0.070
전기차            0.2147      0.015     14.271      0.000       0.185       0.244
변압기           -0.0047      0.004     -1.161      0.246      -0.013       0.003
충전소           28.4331      0.138    206.076      0.000      28.163      28.704
==============================================================================
Omnibus:                    37767.938   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        597409620.588
Skew:                          13.899   Prob(JB):                         0.00
Kurtosis:                     853.742   Cond. No.                     5.12e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
df_pre2.head(1)

,gid,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량,서비스기간,서비스빈도,서비스일수,총/기간,총/빈도,총/일수
0,가사46aa99bb,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Y : 평균충전량(빈도 기준)

In [49]:
df_reg3 = df_pre2.iloc[:, [15,1,2,3,4,5,6,7,8,9]]
df_reg3.head(1)

,총/빈도,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
0,0.0,0,0,0,0,0.0,0,0,0,0.0


In [50]:
df_reg3.rename( columns = {
    '총/빈도' : '평균'
}, inplace = True)
df_reg3.head(1)

,평균,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
0,0.0,0,0,0,0,0.0,0,0,0,0.0


In [51]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model3 = smf.ols(formula = '평균 ~ 건물1 + 건물2 + 건물3 + 관공서 + 경제인구 + 주차장 + 전기차 \
                  + 변압기 + 충전소', data = df_reg3)

result3 = model3.fit()
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     평균   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     9244.
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        09:21:56   Log-Likelihood:                -19974.
No. Observations:               19789   AIC:                         3.997e+04
Df Residuals:                   19779   BIC:                         4.005e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0056      0.005      1.108      0.268      -0.004       0.016
건물1           -0.0003      0.000     -0.521      0.602      -0.001       0.001
건물2           -0.0076      0.004     -2.078      0.038      -0.015      -0.000
건물3            0.0095      0.027      0.346      0.730      -0.044       0.063
관공서            0.0170      0.017      0.995      0.320      -0.017       0.051
경제인구         1.05e-06   2.27e-05      0.046      0.963   -4.34e-05    4.55e-05
주차장            0.0030      0.015      0.203      0.839      -0.026       0.032
전기차            0.0341      0.007      5.239      0.000       0.021       0.047
변압기           -0.0020      0.002     -1.173      0.241      -0.005       0.001
충전소           16.5708      0.060    277.523      0.000      16.454      16.688
==============================================================================
Omnibus:                    16822.613   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        158742943.089
Skew:                          -2.402   Prob(JB):                         0.00
Kurtosis:                     441.748   Cond. No.                     5.12e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
df_pre2.head(1)

,gid,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량,서비스기간,서비스빈도,서비스일수,총/기간,총/빈도,총/일수
0,가사46aa99bb,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Y : 평균충전량(기간 기준)

In [52]:
df_reg4 = df_pre2.iloc[:, [14,1,2,3,4,5,6,7,8,9]]
df_reg4.head(1)

,총/기간,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
0,0.0,0,0,0,0,0.0,0,0,0,0.0


In [53]:
df_reg4.rename( columns = {
    '총/기간' : '평균'
}, inplace = True)
df_reg4.head(1)

,평균,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
0,0.0,0,0,0,0,0.0,0,0,0,0.0


In [54]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model4 = smf.ols(formula = '평균 ~ 건물1 + 건물2 + 건물3 + 관공서 + 경제인구 + 주차장 + 전기차 \
                  + 변압기 + 충전소', data = df_reg4)

result4 = model4.fit()
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     평균   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.472
Method:                 Least Squares   F-statistic:                     1966.
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        09:23:14   Log-Likelihood:                -35652.
No. Observations:               19789   AIC:                         7.132e+04
Df Residuals:                   19779   BIC:                         7.140e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0080      0.011      0.718      0.473      -0.014       0.030
건물1            0.0004      0.001      0.368      0.713      -0.002       0.003
건물2           -0.0178      0.008     -2.191      0.028      -0.034      -0.002
건물3            0.0692      0.061      1.143      0.253      -0.049       0.188
관공서           -0.0353      0.038     -0.934      0.350      -0.109       0.039
경제인구          -0.0001   5.01e-05     -2.567      0.010      -0.000   -3.04e-05
주차장            0.0145      0.033      0.440      0.660      -0.050       0.079
전기차            0.2386      0.014     16.595      0.000       0.210       0.267
변압기           -0.0038      0.004     -0.991      0.322      -0.011       0.004
충전소           16.5633      0.132    125.611      0.000      16.305      16.822
==============================================================================
Omnibus:                    47499.130   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1194353014.605
Skew:                          24.398   Prob(JB):                         0.00
Kurtosis:                    1205.549   Cond. No.                     5.12e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [124]:
sm.add_constant(X).head(1)

,const,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기
0,1.0,0,0,0,0,0.0,0,0,0


# VIF 확인!

In [58]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    df_reg.values, i) for i in range(df_reg.shape[1])]
vif["features"] = df_reg.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif

,VIF Factor,features
0,1.044526,건물3
1,1.139090,주차장
2,1.162814,전기차
3,1.207659,관공서
4,1.846818,변압기
5,1.895798,건물1
6,1.925994,총충전량
7,1.964331,충전소
8,3.181820,건물2
9,3.773818,경제인구


In [57]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif3 = pd.DataFrame()
vif3["VIF Factor"] = [variance_inflation_factor(
    df_reg3.values, i) for i in range(df_reg3.shape[1])]
vif3["features"] = df_reg3.columns
vif3 = vif.sort_values("VIF Factor").reset_index(drop=True)
vif3

,VIF Factor,features
0,1.044519,건물3
1,1.139090,주차장
2,1.144452,전기차
3,1.207666,관공서
4,1.846783,변압기
5,1.895808,건물1
6,3.181229,건물2
7,3.773286,경제인구
8,5.233647,평균
9,5.301522,충전소


In [65]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif4 = pd.DataFrame()
vif4['VIF Factor'] = [variance_inflation_factor(
    df_reg4.values, i) for i in range(df_reg4.shape[1])]
vif4['features'] = df_reg4.columns
vif4 = vif.sort_values('VIF Factor').reset_index(drop = True)
vif4

,VIF Factor,features
0,1.044526,건물3
1,1.139090,주차장
2,1.162814,전기차
3,1.207659,관공서
4,1.846818,변압기
5,1.895798,건물1
6,1.925994,총충전량
7,1.964331,충전소
8,3.181820,건물2
9,3.773818,경제인구


In [66]:
df_pre2.head(1)

,gid,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소,총충전량,서비스기간,서비스빈도,서비스일수,총/기간,총/빈도,총/일수
0,가사46aa99bb,0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
X = df_pre2.iloc[:, 1:9]
X.head(1)

,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기
0,0,0,0,0,0.0,0,0,0


In [80]:
y = df_pre2.iloc[:,16]
y.head(1)

0    0.0
Name: 총/일수, dtype: float64

In [81]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35)
lasso = Lasso(alpha = 0.01, max_iter = 100000).fit(X_train, y_train)
print( "Train Data Score : ",round(lasso.score(X_train, y_train),3), "\n")
print( "Test Data Score : ", round(lasso.score(X_test, y_test),3), "\n")

Train Data Score :  0.069 

Test Data Score :  0.033 



In [82]:
X.head(1)

,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기
0,0,0,0,0,0.0,0,0,0


In [83]:
lasso.coef_

array([-0.01063279, -0.032097  ,  0.01904367,  0.06728361,  0.00132363,
       -0.        ,  0.68952758, -0.        ])

In [84]:
lasso.intercept_

0.05777236211223305

In [144]:
X_2 = X.iloc[:, [0,1,2,3,4,6]]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_2, y, test_size = 0.35)
lasso = Lasso(alpha = 0.01, max_iter = 100000).fit(X_train2, y_train2)
print( "Train Data Score : ",round(lasso.score(X_train2, y_train2),3), "\n")
print( "Test Data Score : ", round(lasso.score(X_test2, y_test2),3), "\n")

Train Data Score :  0.08 

Test Data Score :  0.007 



In [145]:
lasso.coef_

array([-5.58351027e-03, -2.14936938e-02,  1.87994703e-02,  2.18864519e-01,
        7.27254680e-04,  1.04412708e+00])

In [146]:
lasso.intercept_

0.03821925570754375

In [59]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif2 = pd.DataFrame()
vif2["VIF Factor"] = [variance_inflation_factor(
    df_reg2.values, i) for i in range(df_reg2.shape[1])]
vif2["features"] = df_reg2.columns
vif2 = vif2.sort_values("VIF Factor").reset_index(drop=True)
vif2

,VIF Factor,features
0,1.044614,건물3
1,1.139087,주차장
2,1.154675,전기차
3,1.207598,관공서
4,1.846782,변압기
5,1.895798,건물1
6,3.181676,건물2
7,3.368027,평균
8,3.409113,충전소
9,3.773756,경제인구


In [75]:
lasso.sparse_coef_

<1x9 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [76]:
lasso.intercept_

0.016749223803258356

In [77]:
lasso.n_iter_

12

In [87]:
np.sum(lasso.coef_ != 0 )

6

In [73]:
#?lasso

In [96]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [110]:
lr.coef_

array([-0.01011194,  0.00899337,  0.36078884,  0.28314867,  0.00085753,
       -0.08543211,  0.63966828,  0.00491741])

In [111]:
lr.intercept_

0.05142136362117161

In [119]:
from mlxtend.feature_selection import SequentialFeatureSelector

feature_selector = SequentialFeatureSelector( LinearRegression(),
           k_features= 3 ,
           forward=True,
           verbose=2,
           scoring='roc_auc',
           cv=4)

In [120]:
feature_selector.k_feature_names_

AttributeError: 'SequentialFeatureSelector' object has no attribute 'k_feature_names_'

In [109]:
?feature_selector

In [108]:
?LinearRegression()

Object `LinearRegression()` not found.


In [150]:
import numpy as np
import warnings
import os
import statsmodels.formula.api as smf
import pandas as pd
import functools
import re
warnings.filterwarnings('ignore')


class stepwise:

    def __init__(self,step,fit_intercept):
        self.step = step
        self.fit_intercept = fit_intercept

    def reduce_concat(self,x, sep=""):
        return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

    def fit(self,data,null_formula,full_formula,response):

        """Linear model designed by forward selection.
        Parameters:
        -----------
        data : pandas DataFrame with all possible predictors and response
        response: string, name of response column in data
        Returns:
        --------
        model: an "optimal" fitted statsmodels linear model
               with an intercept
               selected by forward selection
               evaluated by aic
        """

        null_temp        = re.split('~',null_formula)
        null_predic_com  = null_temp[1].split('+')
        null_predic      = null_predic_com[1:len(null_predic_com)]
        full_temp        = re.split('~',full_formula)
        full_predic_com  = full_temp[1].split('+')
        full_predic      = full_predic_com[1:len(full_predic_com)]
        indices          = [i for i,id in enumerate(full_predic) if id not in null_predic]
        domain           = [full_predic[i] for i in indices]
        start            = set(null_predic)
        remaining        = set(domain)
        selected         = null_predic
        current_score, best_new_score = float('inf'), float('inf')
        score_selected   = []
        variable_added   = []
        
        while (remaining and current_score == best_new_score and self.step >0):
            scores_with_candidates = []
            for candidate in remaining:
                formula = "{} ~ {}".format(response,' + '.join(selected + [candidate]))
                if self.fit_intercept == 0:
                    formula = formula + "-1"
                score = smf.ols(formula, data).fit().aic
                scores_with_candidates.append((score, candidate))
            scores_with_candidates.sort()
            best_new_score, best_candidate = scores_with_candidates.pop(0)
            if current_score > best_new_score:
                remaining.remove(best_candidate)
                selected.append(best_candidate)
                score_selected.append(best_new_score)
                variable_added.append(best_candidate)
                current_score = best_new_score
            self.step=self.step-1
        formula = "{} ~ {}".format(response,' + '.join(selected))
        if self.fit_intercept == 0:
            formula = formula + "-1"
        model = smf.ols(formula, data).fit()
        return model



In [152]:
from distutils.core import setup

setup(
    name              = 'stepwisereg',
    packages          = ['stepwisereg'],
    version           = '0.1.0',
    description       = 'Stepwise Regression in Python',
    long_description  = 'Forward Stepwise Regression in Python like R using AIC',
    author = 'Avinash Barnwal',
    author_email = 'avinashbarnwal123@gmail.com',
    url = 'https://github.com/avinashbarnwal/stepwisereg',
    maintainer='Avinash Barnwal',
    maintainer_email='avinashbarnwal123@gmail.com',
    install_requires=[
    'numpy',
    'pandas',
 'sklearn',
 'patsy',
 'statsmodel',
 'functools',
 're'
    ],
    download_url = 'https://github.com/avinashbarnwal/stepwisereg/tarball/0.1.11',
    keywords = ['stepwise', 'python3', 'sklearn','regression'],
    license='The MIT License (MIT)',
    classifiers = ["Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",],
)

SystemExit: usage: ipykernel_launcher.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: ipykernel_launcher.py --help [cmd1 cmd2 ...]
   or: ipykernel_launcher.py --help-commands
   or: ipykernel_launcher.py cmd --help

error: option -f not recognized

# Stepwise

In [125]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model3 = smf.ols(formula = '평균 ~ 건물1 + 건물2 + 건물3 + 관공서 + 경제인구 + 주차장 + 전기차 \
                  + 변압기 + 충전소', data = df_reg3)

result3 = model3.fit()
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     평균   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     9244.
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        15:52:25   Log-Likelihood:                -19974.
No. Observations:               19789   AIC:                         3.997e+04
Df Residuals:                   19779   BIC:                         4.005e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0056      0.005      1.108      0.268      -0.004       0.016
건물1           -0.0003      0.000     -0.521      0.602      -0.001       0.001
건물2           -0.0076      0.004     -2.078      0.038      -0.015      -0.000
건물3            0.0095      0.027      0.346      0.730      -0.044       0.063
관공서            0.0170      0.017      0.995      0.320      -0.017       0.051
경제인구         1.05e-06   2.27e-05      0.046      0.963   -4.34e-05    4.55e-05
주차장            0.0030      0.015      0.203      0.839      -0.026       0.032
전기차            0.0341      0.007      5.239      0.000       0.021       0.047
변압기           -0.0020      0.002     -1.173      0.241      -0.005       0.001
충전소           16.5708      0.060    277.523      0.000      16.454      16.688
==============================================================================
Omnibus:                    16822.613   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        158742943.089
Skew:                          -2.402   Prob(JB):                         0.00
Kurtosis:                     441.748   Cond. No.                     5.12e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [140]:
df_reg3.head(1)

,평균,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
0,0.0,0,0,0,0,0.0,0,0,0,0.0


In [141]:
df_reg3_1 = df_reg3.iloc[: , [0,1,2,3,4,6,7,8,9]]

model3_1 = smf.ols(formula = '평균 ~ 건물1 + 건물2 + 건물3 + 관공서 + 주차장 + 전기차 \
                  + 변압기 + 충전소', data = df_reg3_1)

result3_1 = model3_1.fit()
result3_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     평균   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                 1.040e+04
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        16:05:19   Log-Likelihood:                -19974.
No. Observations:               19789   AIC:                         3.997e+04
Df Residuals:                   19780   BIC:                         4.004e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0056      0.005      1.111      0.266      -0.004       0.016
건물1           -0.0003      0.000     -0.520      0.603      -0.001       0.001
건물2           -0.0075      0.002     -3.228      0.001      -0.012      -0.003
건물3            0.0095      0.027      0.345      0.730      -0.044       0.063
관공서            0.0171      0.017      1.012      0.311      -0.016       0.050
주차장            0.0031      0.015      0.213      0.831      -0.026       0.032
전기차            0.0341      0.006      5.280      0.000       0.021       0.047
변압기           -0.0020      0.002     -1.172      0.241      -0.005       0.001
충전소           16.5711      0.059    278.595      0.000      16.454      16.688
==============================================================================
Omnibus:                    16823.260   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        158755600.644
Skew:                          -2.402   Prob(JB):                         0.00
Kurtosis:                     441.765   Cond. No.                         174.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [134]:
df_reg3_2 = df_reg3.iloc[: , [0,1,2,3,4,7,8,9]]

model3_2 = smf.ols(formula = '평균 ~ 건물1 + 건물2 + 건물3 + 관공서 + 전기차 \
                  + 변압기 + 충전소', data = df_reg3_2)

result3_2 = model3_2.fit()
result3_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     평균   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                 1.189e+04
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        16:00:21   Log-Likelihood:                -19974.
No. Observations:               19789   AIC:                         3.996e+04
Df Residuals:                   19781   BIC:                         4.003e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0056      0.005      1.113      0.266      -0.004       0.016
건물1           -0.0002      0.000     -0.488      0.626      -0.001       0.001
건물2           -0.0075      0.002     -3.221      0.001      -0.012      -0.003
건물3            0.0094      0.027      0.341      0.733      -0.044       0.063
관공서            0.0174      0.017      1.028      0.304      -0.016       0.050
전기차            0.0342      0.006      5.286      0.000       0.022       0.047
변압기           -0.0020      0.002     -1.171      0.242      -0.005       0.001
충전소           16.5711      0.059    278.602      0.000      16.454      16.688
==============================================================================
Omnibus:                    16823.294   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        158744661.384
Skew:                          -2.402   Prob(JB):                         0.00
Kurtosis:                     441.750   Cond. No.                         174.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [137]:
df_reg3.head(1)

,평균,건물1,건물2,건물3,관공서,경제인구,주차장,전기차,변압기,충전소
0,0.0,0,0,0,0,0.0,0,0,0,0.0


In [135]:
df_reg3_3 = df_reg3.iloc[: , [0,1,2,4,7,8,9]]

model3_3 = smf.ols(formula = '평균 ~ 건물1 + 건물2 + 관공서 +  전기차 \
                  + 변압기 + 충전소', data = df_reg3_3)

result3_3 = model3_3.fit()
result3_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     평균   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                 1.387e+04
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        16:00:50   Log-Likelihood:                -19974.
No. Observations:               19789   AIC:                         3.996e+04
Df Residuals:                   19782   BIC:                         4.002e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0056      0.005      1.115      0.265      -0.004       0.016
건물1           -0.0002      0.000     -0.483      0.629      -0.001       0.001
건물2           -0.0075      0.002     -3.227      0.001      -0.012      -0.003
관공서            0.0180      0.017      1.071      0.284      -0.015       0.051
전기차            0.0344      0.006      5.341      0.000       0.022       0.047
변압기           -0.0020      0.002     -1.154      0.248      -0.005       0.001
충전소           16.5715      0.059    278.672      0.000      16.455      16.688
==============================================================================
Omnibus:                    16819.143   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        158726804.192
Skew:                          -2.401   Prob(JB):                         0.00
Kurtosis:                     441.725   Cond. No.                         174.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [139]:
df_reg3_4 = df_reg4.iloc[: , [0,2,4,7,8,9]]

model3_4 = smf.ols(formula = '평균 ~  건물2 + 관공서 +  전기차 + 변압기 + 충전소', data = df_reg3_4)

result3_4 = model3_4.fit()
result3_4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     평균   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.472
Method:                 Least Squares   F-statistic:                     3537.
Date:                Wed, 06 Nov 2019   Prob (F-statistic):               0.00
Time:                        16:02:59   Log-Likelihood:                -35656.
No. Observations:               19789   AIC:                         7.132e+04
Df Residuals:                   19783   BIC:                         7.137e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0070      0.011      0.633      0.527      -0.015       0.029
건물2           -0.0340      0.005     -6.641      0.000      -0.044      -0.024
관공서           -0.0446      0.037     -1.208      0.227      -0.117       0.028
전기차            0.2359      0.014     16.600      0.000       0.208       0.264
변압기           -0.0042      0.003     -1.349      0.177      -0.010       0.002
충전소           16.5368      0.131    125.964      0.000      16.280      16.794
==============================================================================
Omnibus:                    47537.074   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1199315874.037
Skew:                          24.447   Prob(JB):                         0.00
Kurtosis:                    1208.045   Cond. No.                         48.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Lasso 사용 변수 제거

In [142]:
result3 = model.fit_regularized(alpha=0.01, L1_wt=1)
print(result3.params)
plot_statsmodels(result3)

Intercept       2.802898
건물1             0.017658
건물2            -5.328411
건물3            -4.770262
관공서           -18.008767
경제인구           -0.039548
주차장             3.192780
전기차           124.527247
변압기            -2.337918
충전소          7803.191081
dtype: float64


NameError: name 'plot_statsmodels' is not defined